## I. 问题的定义

### 项目概述

　　汽车的发明给人类社会带来了巨大的发展，人们的出行变得方便快捷，但随之也带来了交通拥堵与交通事故等问题。而随着计算机技术的发展，越来越多的自动控制技术被应用在汽车上，无人驾驶汽车也成为了汽车产业的一大变革。早在20世纪80年代，美国就提出自主地面车辆 (ALV)计划，这是一辆8轮车，能在校园的环境中自主驾驶，但车速不高。世界各大强国也在这方面投入资源。
 
　　近年来，随着深度学习的发展，无人驾驶发展非常迅速。其中比较具代表性的分别是Mobileye的自动驾驶布局，英伟达的DRIVE解决方案以及Commaai的方法，后两者都有采用的端到端方法，指以摄像头的原始图像作为输入，中间通过前期使用数据训练出来的模型直接输出车辆的速度和方向。[1]
  
　　本项目基于"MIT 6.S094: Deep Learning for Self-Driving Cars"。在该项目中，我们利用MIT 6.S094 这门公开课中的Tesla行驶数据集中前置相机所拍摄的路况图像，训练深度学习模型，利用该模型对车辆转向角度的预测。[2]


### 问题陈述

　　该项目是一个监督回归问题。训练数据集中对应每一帧图像都给定了相应的方向盘转向角度。此处使用端到端(end-to-end)模型，端到端模型指的是输入是原始数据，输出是最后的结果。在本问题中，输入X是前置摄像头获取的单帧图片，而最后的结果Y为转向角度，我们的目标就是通过训练学习到一个端到端模型f，利用该模型则可以预测转向角度Y，其关系为Y=f(X)。

### 评价指标

　　该项目是回归问题，该问题的评价指标主要有两个，预测结果平均损失函数(MSE)以及模型的训练和预测时长。MSE表达式如下，其中 $y_{p}$ 表示预测转向角度，y为实际转向角度，N为样本总数。MSE值越小，则模型效果越好。

$MSE=\dfrac {1} {N}\sum _{i=1}^{N}{(y_{p}-y)}^2$

## II. 分析

### 数据的探索

　　先来看一下项目的输入，项目的原始数据来处于汽车的前置摄像头，以mkv的视频格式提供的，共10段视频，其中第10段视频将用于测试，第1到第9个视频经提取后，共有24300张图片，图片形状为（720，1200，3）。


　　从第1个视频中随机获取9张图如下，我们可以看到，照片中会有大量无用区域(如天空，车辆，相机反光等)，在预处理时需要把这些部分去掉，可以压缩训练数据加快模型训练。
  
![](images/notebook_img/img1.png)

　　再来看看项目输出，其中，ts_micro表示时间戳，frame_index表示帧编号，wheel表示方向盘转向角度（以水平方向为基准，+为顺时针，-为逆时针）
输出

![](images/notebook_img/img2.jpg)

### 探索性可视化

　　将第一段视频的输出汇总成柱状图和折线图如下。从这两个图可以看到，分布在0附近的信号较多，有可能会对大转变不够敏感，但总体而言数据的分布差距不是很大，模型可以学到有价值的转身策略。
  
![](images/notebook_img/img3.png)
![](images/notebook_img/img4.png)

### 算法和技术

　　该项目是一个监督回归问题。训练数据集中对应每一帧图像都给定了相应的方向盘转向角度。此处使用端到端(end-to-end)模型，端到端模型指的是输入是原始数据，输出是最后的结果。在本问题中，输入X是前置摄像头获取的单帧图片，而最后的结果Y为转向角度，我们的目标就是通过训练学习到一个端到端模型f，利用该模型则可以预测转向角度Y，其关系为Y=f(X)。

　　由于具有图片状态空间巨大、图像含义在不同位置的平移不变性(translation invariance)等特点，传统的监督学习算法并不适用，目前最适合于图片检测的算法是卷积神经网络(CNN)。卷积神经网络不同层之间的神经元并不是全连接，而是采用局部连接的方式，这使得网络上的参数大大减少，并且共享权重，具有很好的物体平移不变性。

　　在训练中，涉及到的参数如下：

- 预处理:图片尺寸(size)，感兴趣区域（region of interest,ROI）
- 超参数:训练次数(epoch),批处理大小(batch size),优化器类型(optimizer type),学习速率(learning rate)
- 神经网络架构:网络层数(layers),网络层类型(layer types)，网络层初始化参数及偏置项(weights and bias)

### 基准模型

　　英伟达（NVIDIA）一直以来发力进行深度学习和无人驾驶的研究，今年年初NVIDIA就发表了一篇论文介绍如何利用CNNs实现端到端的无人驾驶[3]。此处采用该模型来作为基准模型。

　　如下图所示，该模型第一层为归一化层，接下来的紧跟着5个卷积层，在卷积层后增加3个全连接层。
　　
![](images/15028711627212.jpg)

## III. 方法

### 数据预处理

数据预处理中，本项目做了以下处理。

先将视频中的所有图片提取出来，并且提取输出结果，使图片与输出一一对应。

由于数据较少(24300)，因为此处先将数据进行亮度调整，数据总量为(24300×2)，最后再将所有数据进行水平翻转，同时将label的值修改(label=-label),得到24,300×4张图片

由于图片尺寸比较大（720，1200，3），并且无用区域较多，此处会对所有图片进行裁剪，只留下感兴趣区域。

### 执行过程

该项目实现了两种模型来测试效果，分别是分别是英伟达提出的端到端卷积神经网络模型和基于迁移学习的VGG16模型。

#### 英伟达端到端卷积神经网络模型

英伟达模型的情况在基准模型章节已经有介绍，此处就不再描述。代码实现如下

#### VGG16模型

VGG 模型是在 LSVRC-14 提出的一系列模型，其中 16 层模型和 19 层模型在 ImageNet 上表现最好，这两个模型在后来就被称作 VGG16 和 VGG19。

VGG 模型的 结构很简单，整个模型由几组相同 filter 的卷积层叠加而成，filter 大小均是 3x3，每一 组卷积层后面叠加一个相同的 MaxPooling 层。[3]

基于 VGG16 模型简洁和直观性，在考虑使用卷积神经网络解决图像问题时，通常可 以先使用 VGG16 来对数据集进行验证，如果 VGG 模型能得到不错的效果，可以再考 虑对模型进行优化或使用层数更多的模型。

在这个实现方案中，我们使用了 Keras 自带的 VGG16 模型，weights 使用 'imagenet' 预 训练 weights。

训练阶段，首先锁住预训练模型的所有层，同时添加3层全连接层。

### 完善
在这一部分，你需要描述你对原有的算法和技术完善的过程。例如调整模型的参数以达到更好的结果的过程应该有所记录。你需要记录最初和最终的模型，以及过程中有代表性意义的结果。你需要考虑的问题：
- _初始结果是否清晰记录了？_
- _完善的过程是否清晰记录了，其中使用了什么技术？_
- _完善过程中的结果以及最终结果是否清晰记录了？_


## IV. 结果
_（大概 2-3 页）_

### 模型的评价与验证
在这一部分，你需要对你得出的最终模型的各种技术质量进行详尽的评价。最终模型是怎么得出来的，为什么它会被选为最佳需要清晰地描述。你也需要对模型和结果可靠性作出验证分析，譬如对输入数据或环境的一些操控是否会对结果产生影响（敏感性分析sensitivity analysis）。一些需要考虑的问题：
- _最终的模型是否合理，跟期待的结果是否一致？最后的各种参数是否合理？_
- _模型是否对于这个问题是否足够稳健可靠？训练数据或输入的一些微小的改变是否会极大影响结果？（鲁棒性）_
- _这个模型得出的结果是否可信？_

1.回答上述三个问题

### 合理性分析
在这个部分，你需要利用一些统计分析，把你的最终模型得到的结果与你的前面设定的基准模型进行对比。你也分析你的最终模型和结果是否确确实实解决了你在这个项目里设定的问题。你需要考虑：
- _最终结果对比你的基准模型表现得更好还是有所逊色？_
- _你是否详尽地分析和讨论了最终结果？_
- _最终结果是不是确确实实解决了问题？_

1.使用各种图去比较上面三个模型的结果，并得出结论

## V. 项目结论
_（大概 1-2 页）_

### 结果可视化
在这一部分，你需要用可视化的方式展示项目中需要强调的重要技术特性。至于什么形式，你可以自由把握，但需要表达出一个关于这个项目重要的结论和特点，并对此作出讨论。一些需要考虑的：
- _你是否对一个与问题，数据集，输入数据，或结果相关的，重要的技术特性进行了可视化？_
- _可视化结果是否详尽的分析讨论了？_
- _绘图的坐标轴，标题，基准面是不是清晰定义了？_

生成的视频描述一下


### 对项目的思考
在这一部分，你需要从头到尾总结一下整个问题的解决方案，讨论其中你认为有趣或困难的地方。从整体来反思一下整个项目，确保自己对整个流程是明确掌握的。需要考虑：
- _你是否详尽总结了项目的整个流程？_
- _项目里有哪些比较有意思的地方？_
- _项目里有哪些比较困难的地方？_
- _最终模型和结果是否符合你对这个问题的期望？它可以在通用的场景下解决这些类型的问题吗？_
1.总结并回答上面4个问题


### 需要作出的改进
在这一部分，你需要讨论你可以怎么样去完善你执行流程中的某一方面。例如考虑一下你的操作的方法是否可以进一步推广，泛化，有没有需要作出变更的地方。你并不需要确实作出这些改进，不过你应能够讨论这些改进可能对结果的影响，并与现有结果进行比较。一些需要考虑的问题：
- _是否可以有算法和技术层面的进一步的完善？_
- _是否有一些你了解到，但是你还没能够实践的算法和技术？_
- _如果将你最终模型作为新的基准，你认为还能有更好的解决方案吗？_
1.回答三个问题